Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

To address the questions below, we will not use third party libraries such as pandas and numpy, and instead focus on what's available in the Python Standard Library.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

Questions to be addressed are listed below:
1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. 

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample =requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2018-03-27&end_date=2018-03-27&api_key='+API_KEY)


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(sample.json())

# Structure:
# dataset_data:   - limit: None
#                 - transform: None
#                 - column_index: None
#                 - column_names: 'Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'
#                 - start_date
#                 - end_date
#                 - frequency: daily
#                 - data

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2018-03-27', 'end_date': '2018-03-27', 'frequency': 'daily', 'data': [['2018-03-27', None, 52.1, 50.35, 51.7, None, 89089.0, 4582469.0, None, None, None]], 'collapse': None, 'order': None}}


In [76]:
# 1. Collect data from the Franfurt Stock Exchange (database FSE), for the ticker AFX_X (dataset AFX_X), for the whole year 2017
r =requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)


In [77]:
# 2. Convert the returned JSON object into a Python dictionary.
afx_x = dict(r.json())

In [126]:
# Inspect data
afx_x['dataset_data']['data'][0:2]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None]]

In [80]:
# Extract key data and store in lists
prices = []
dates = []
high = []
low = []
close = []
volume = []
for i in afx_x['dataset_data']['data']:
    if i[1] is None: continue  # Remove None values
    # Or use dictionary comprehension later: newdict = {k: v for k, v in newdict.items() if v is not None}
    prices.append(i[1]) 
    dates.append(i[0])
    high.append(i[2])
    low.append(i[3])
    close.append(i[4])
    volume.append(i[6])

In [64]:
# 3. Getting max and min open price and date
newdict = dict(zip(dates, prices))

max_open = max(newdict.values())
max_open_date = [k for k, v in newdict.items() if v== max_open]
min_open = min(newdict.values())
min_open_date = [k for k, v in newdict.items() if v== min_open]
print('The date with maximum open price is {} and the price is {}'.format(max_open_date[0], max_open))
print('The date with minimum open price is {} and the price is {}'.format(min_open_date[0], min_open))

The date with maximum open price is 2017-12-14 and the price is 53.11
The date with minimum open price is 2017-01-24 and the price is 34.0


In [65]:
# If convert to datetime
from datetime import datetime
print(datetime.strptime(max_open_date[0], '%Y-%m-%d'))


2017-12-14 00:00:00


In [113]:
# 4. What was the largest change in any one day (based on High and Low price)?
# Use list comprehension to calculate price change in each day
diff = [x-y for x, y in zip(high,low)]

diff_dict = dict(zip(dates,diff))

largest_diff_date = [k for k, v in diff_dict.items() if v == max(diff)]
print('The largest change in any one day is {} on {}'.format(round(max(diff),2), largest_diff_date[0]))


The largest change in any one day is 2.81 on 2017-05-11


In [66]:
# 5. What was the largest change between any two days (based on Closing Price)?

# Calculate the largest change
change = max(close)-min(close)
close_dict = dict(zip(dates,close))

max_close = [k for k, v in close_dict.items() if v == max(close)]
min_close = [k for k, v in close_dict.items() if v == min(close)]

print('The largest change between any two days is {} between {} and {}'.format(change, max_close[0], min_close[0]))


The largest change between any two days is 19.03 between 2017-12-13 and 2017-01-23


In [114]:
# 6. What was the average daily trading volume during this year?
average = sum(volume)/len(volume)
print('The average daily trading volume is', round(average,2))


The average daily trading volume is 89140.98


In [125]:
# (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def median(x):
    sort =sorted(x)
    if len(sort) % 2 == 0: 
        return (sort[int(len(sort)/2-1)] + sort[int(len(sort)/2)])/2
    else:
        return sort[int((len(sort)+1)/2)]

median_volume= median(volume)
print('The median trading volume during this year is',median_volume)

The median trading volume during this year is 74723.5


In [115]:
# Double check with function median() from the statistics package:
import statistics
statistics.median(volume)

74723.5